In [1]:
# %cd ./TextRecognitionDataGenerator/

In [6]:
# !pip install .

In [5]:
# %cd ../

In [402]:
from glob import glob
import os
import pandas as pd
import random
import numpy as np
from random import randrange
from tqdm import tqdm
from datetime import timedelta
from datetime import datetime
from PIL import Image


from trdg.generators import (
    GeneratorFromDict,
    GeneratorFromRandom,
    GeneratorFromStrings,
    GeneratorFromWikipedia,
)

In [12]:
df = pd.read_csv('./TextRecognitionDataGenerator/trdg/dicts/ka.txt', header=None)
geo_words = df.iloc[:, 0].values.tolist()

In [389]:
text_symbols = [',', '.', '!', '?', '']

def get_words():
    word_number = np.random.randint(low=2, high=6, size=1).item()
    rand_symbol = random.sample(text_symbols, k=1)
    word_list = random.sample(geo_words, k=word_number)
    break_index = np.random.randint(low=0, high=len(word_list), size=1).item()
    first_part = word_list[:break_index]
    second_part = word_list[break_index:]
    words = ' '.join(first_part) + rand_symbol[0] + ' ' +  ' '.join(second_part)
    words = words.strip()
    if words[0] in text_symbols:
        return words[2:]
    return words

In [390]:
get_words()

'ათუალვა შეწირულობის ვარაყშეყრილი'

In [391]:
def random_date(start, end):
    """
    This function will return a random datetime between two datetime 
    objects.
    """
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)


start = datetime.strptime('01/01/2001', '%d/%m/%Y')
end = datetime.strptime('16/11/2021', '%d/%m/%Y')

def get_date():
    date = random_date(start, end).strftime("%d.%m.%Y")
    return date

In [392]:
get_date()

'28.09.2013'

In [393]:
math_symbols = ['', '%', '$', '.', ',']

def numbers():
    rand_symbol = random.sample(math_symbols, k=1)
    numbers = np.random.randint(low=0, high=10000)
    numbers = str(numbers) + rand_symbol[0]
    if numbers[-1] == '.':
        numbers += str(np.random.randint(low=0, high=100))

    return numbers

In [394]:
def get_string(how_many: int, gen_words_dates_numbers: tuple = (0.6, 0.2, 0.2)):
    strings = []
    
    while not len(strings) == how_many:
        random_number = np.random.random()
        text = None
        if random_number < sum(gen_words_dates_numbers[:1]):
            text = get_words()
        elif random_number > sum(gen_words_dates_numbers[:1]) and random_number < sum(gen_words_dates_numbers[:2]):
            text = get_date()
        else:
            text = numbers()
        strings.append(text)
        
    return strings

In [400]:
get_string(5)

['11.04.2001',
 'დაარბილა? კვანძური მოისპობა პატრულირებული დრომ',
 '1025$',
 'უპრიანია ბელგარდი',
 'საცერები ბრილის']

# First genrate white background images

In [4]:
GeneratorFromStrings?

Init signature:
GeneratorFromStrings(
    strings,
    count=-1,
    fonts=[],
    language='en',
    size=32,
    skewing_angle=0,
    random_skew=False,
    blur=0,
    random_blur=False,
    background_type=0,
    distorsion_type=0,
    distorsion_orientation=0,
    is_handwritten=False,
    width=-1,
    alignment=1,
    text_color='#282828',
    orientation=0,
    space_width=1.0,
    character_spacing=0,
    margins=(5, 5, 5, 5),
    fit=False,
    output_mask=False,
    word_split=False,
    image_dir='/home/ec2-user/SageMaker/ocr/TextRecognitionDataGenerator/trdg/generators/images',
    stroke_width=0,
    stroke_fill='#282828',
    image_mode='RGB',
)
Docstring:      Generator that uses a given list of strings
File:           ~/SageMaker/ocr/TextRecognitionDataGenerator/trdg/generators/from_strings.py
Type:           type
Subclasses:     


train = 200000
val = 3000
test = 3000

In [460]:
training_strings = get_string(30_000)
val_strings = get_string(1500)
test_strings = get_string(1500)

In [461]:
fonts_pathes = glob('./TextRecognitionDataGenerator/trdg/fonts/ka/*')
fonts_pathes

['./TextRecognitionDataGenerator/trdg/fonts/ka/bpg_nino_mtavruli_normal.ttf',
 './TextRecognitionDataGenerator/trdg/fonts/ka/arial_geo.ttf',
 './TextRecognitionDataGenerator/trdg/fonts/ka/bpg_glaho_sylfaen.ttf',
 './TextRecognitionDataGenerator/trdg/fonts/ka/bpg_mrgvlovani_caps_2010.ttf',
 './TextRecognitionDataGenerator/trdg/fonts/ka/bpg_nino_mtavruli_bold.ttf',
 './TextRecognitionDataGenerator/trdg/fonts/ka/bpg_glaho.ttf']

# Generate Train

In [453]:
# The generators use the same arguments as the CLI, only as parameters
train_generator = GeneratorFromStrings(
    strings=training_strings,
    fonts=fonts_pathes,
    language='ka',
    skewing_angle=5,
    random_skew=True,
    blur=1,
    random_blur=True,
    background_type=1,
)

In [463]:
import os

os.makedirs('train_white', exist_ok=True)

for i, (img, lbl) in tqdm(enumerate(train_generator)):
    image_path = os.path.join('train_white', f'{lbl}_{i}.jpg')
    img.save(image_path)
    
    if i == len(training_strings):
        break
    

31023it [30:25, 17.00it/s]


KeyboardInterrupt: 

# Generate Test

In [464]:
# The generators use the same arguments as the CLI, only as parameters
test_generator = GeneratorFromStrings(
    strings=test_strings,
    fonts=fonts_pathes,
    language='ka',
    skewing_angle=5,
    random_skew=True,
    blur=1,
    random_blur=True,
    background_type=1,
)

In [465]:
os.makedirs('test_white', exist_ok=True)

for i, (img, lbl) in tqdm(enumerate(test_generator)):
    image_path = os.path.join('test_white', f'{lbl}_{i}.jpg')
    img.save(image_path)
    
    if i == len(test_strings):
        break

1500it [02:22, 10.51it/s]


# Generate val

In [466]:
# The generators use the same arguments as the CLI, only as parameters
val_generator = GeneratorFromStrings(
    strings=val_strings,
    fonts=fonts_pathes,
    language='ka',
    skewing_angle=5,
    random_skew=True,
    blur=1,
    random_blur=True,
    background_type=1,
)

In [467]:
os.makedirs('val_white', exist_ok=True)

for i, (img, lbl) in tqdm(enumerate(val_generator)):
    image_path = os.path.join('val_white', f'{lbl}_{i}.jpg')
    img.save(image_path)
    
    if i == len(val_strings):
        break

1500it [00:40, 37.03it/s]


# Now generate black backgorund images

In [487]:
training_strings = get_string(30_000)
val_strings = get_string(1500)
test_strings = get_string(1500)

### Train images

In [488]:
# The generators use the same arguments as the CLI, only as parameters
train_generator = GeneratorFromStrings(
    strings=training_strings,
    fonts=fonts_pathes,
    language='ka',
    skewing_angle=5,
    random_skew=True,
    blur=1,
    random_blur=True,
    background_type=3,
    text_color='#ffffff',
    stroke_fill='#ffffff'
)

In [493]:
import os

os.makedirs('train_black', exist_ok=True)

for i, (img, lbl) in tqdm(enumerate(train_generator)):
    image_path = os.path.join('train_black', f'{lbl}_{i}.jpg')
    try:
        img.save(image_path)
    except AttributeError:
        print('cannot save image:', i)
        
    if i == len(training_strings):
        break
    

170it [00:05, 35.05it/s]

value of mean pixel is too similar. Ignore this image
resized_img_st 
 [255.0, 255.0, 255.0, 192.83878504672896]
background_img_st 
 [70.38741134751773, 70.38741134751773, 70.38741134751773, 255.0]
cannot save image: 163


261it [00:07, 33.04it/s]

value of mean pixel is too similar. Ignore this image
resized_img_st 
 [255.0, 255.0, 255.0, 210.33576642335765]
background_img_st 
 [71.74408783783784, 71.74408783783784, 71.74408783783784, 255.0]
cannot save image: 255


307it [00:09, 32.21it/s]

...

29734it [33:20,  8.83it/s]

value of mean pixel is too similar. Ignore this image
resized_img_st 
 [255.0, 255.0, 255.0, 194.30701754385964]
background_img_st 
 [72.03988486842105, 72.03988486842105, 72.03988486842105, 255.0]
cannot save image: 29734


29778it [33:24, 14.74it/s]

value of mean pixel is too similar. Ignore this image
resized_img_st 
 [255.0, 255.0, 255.0, 218.83904109589042]
background_img_st 
 [70.3828125, 70.3828125, 70.3828125, 255.0]
cannot save image: 29775


30000it [33:49, 14.78it/s]


In [494]:
!ls train_black/ | wc -l

30977


## Val images

In [495]:
# The generators use the same arguments as the CLI, only as parameters
val_generator = GeneratorFromStrings(
    strings=val_strings,
    fonts=fonts_pathes,
    language='ka',
    skewing_angle=5,
    random_skew=True,
    blur=1,
    random_blur=True,
    background_type=3,
    text_color='#ffffff',
    stroke_fill='#ffffff'
)

In [497]:
import os

os.makedirs('val_black', exist_ok=True)

for i, (img, lbl) in tqdm(enumerate(train_generator)):
    image_path = os.path.join('val_black', f'{lbl}_{i}.jpg')
    try:
        img.save(image_path)
    except AttributeError:
        print('cannot save image:', i)
    
    if i == len(val_strings):
        break
    

35it [00:00, 37.36it/s]

value of mean pixel is too similar. Ignore this image
resized_img_st 
 [255.0, 255.0, 255.0, 206.98828125]
background_img_st 
 [72.34801136363636, 72.34801136363636, 72.34801136363636, 255.0]
cannot save image: 33


66it [00:01, 38.51it/s]

value of mean pixel is too similar. Ignore this image
resized_img_st 
 [255.0, 255.0, 255.0, 205.70224719101122]
background_img_st 
 [70.77320544554455, 70.77320544554455, 70.77320544554455, 255.0]
cannot save image: 56


91it [00:02, 38.00it/s]

...

1091it [00:33, 29.93it/s]

value of mean pixel is too similar. Ignore this image
resized_img_st 
 [255.0, 255.0, 255.0, 198.9633507853403]
background_img_st 
 [74.91045673076923, 74.91045673076923, 74.91045673076923, 255.0]
cannot save image: 1086


1416it [00:43, 36.19it/s]

value of mean pixel is too similar. Ignore this image
resized_img_st 
 [255.0, 255.0, 255.0, 198.18823529411765]
background_img_st 
 [70.9116847826087, 70.9116847826087, 70.9116847826087, 255.0]
cannot save image: 1412


1500it [00:46, 32.29it/s]


## Test images

In [498]:
# The generators use the same arguments as the CLI, only as parameters
test_generator = GeneratorFromStrings(
    strings=test_strings,
    fonts=fonts_pathes,
    language='ka',
    skewing_angle=5,
    random_skew=True,
    blur=1,
    random_blur=True,
    background_type=3,
    text_color='#ffffff',
    stroke_fill='#ffffff'
)

In [499]:
import os

os.makedirs('test_black', exist_ok=True)

for i, (img, lbl) in tqdm(enumerate(train_generator)):
    image_path = os.path.join('test_black', f'{lbl}_{i}.jpg')
    try:
        img.save(image_path)
    except AttributeError:
        print('cannot save image:', i)
    
    if i == len(test_strings):
        break
    

76it [00:02, 34.77it/s]

value of mean pixel is too similar. Ignore this image
resized_img_st 
 [255.0, 255.0, 255.0, 202.45426829268294]
background_img_st 
 [71.12916666666666, 73.134375, 70.00729166666666, 255.0]
cannot save image: 70


319it [00:09, 31.25it/s]

value of mean pixel is too similar. Ignore this image
resized_img_st 
 [255.0, 255.0, 255.0, 211.59756097560975]
background_img_st 
 [71.55487204724409, 71.55487204724409, 71.55487204724409, 255.0]
cannot save image: 314


336it [00:10, 30.43it/s]

...

1466it [00:45, 32.28it/s]

value of mean pixel is too similar. Ignore this image
resized_img_st 
 [253.96341463414635, 253.96341463414635, 253.96341463414635, 196.16260162601625]
background_img_st 
 [71.50847457627118, 71.50847457627118, 71.50847457627118, 255.0]
cannot save image: 1462


1486it [00:46, 31.30it/s]

value of mean pixel is too similar. Ignore this image
resized_img_st 
 [255.0, 255.0, 255.0, 194.45982142857142]
background_img_st 
 [70.81298828125, 72.7861328125, 69.75390625, 255.0]
cannot save image: 1481


1500it [00:46, 32.03it/s]


Move images into one gib folder

In [500]:
# !mkdir ./data
# !mkdir ./data/train
# !mkdir ./data/test
# !mkdir ./data/val

In [512]:
# !for i in ./train_white/*; do cp "$i" ./data/train/; done
# !for i in ./train_black/*; do cp "$i" ./data/train/; done
!ls ./data/train | wc -l

62000


In [513]:
# !for i in ./test_white/*; do cp "$i" ./data/test/; done
# !for i in ./test_black/*; do cp "$i" ./data/test/; done
!ls ./data/test | wc -l

2989


In [514]:
# !for i in ./val_white/*; do cp "$i" ./data/val/; done
# !for i in ./val_black/*; do cp "$i" ./data/val/; done
!ls ./data/val | wc -l

2999


In [ ]:
# !zip -r data.zip ./data